In [ ]:
import os, sys, glob
import matplotlib.pyplot as plt
import numpy as np
import cv2
from PIL import Image

import torch
from torchsummary import summary
from models import *
from torchsummary import summary
import torchvision
from torchvision import transforms, utils

In [ ]:
!pip install yacs

In [1]:
from yacs.config import CfgNode as CN

ModuleNotFoundError: No module named 'yacs'

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=0., std=1.)
])

In [ ]:
img = Image.open('../car.jpg')

In [ ]:
# ResNet Model
model = torchvision.models.resnet50(pretrained=True)

# model.load_state_dict(torch.load('../models/ResNet_adamx.pth'))

In [ ]:
# we will save the conv layer weights in this list
model_weights =[]
#we will save the 49 conv layers in this list
conv_layers = []
# get all the model children as list
model_children = list(model.children())
#counter to keep count of the conv layers
counter = 0
#append all the conv layers and their respective wights to the list
for i in range(len(model_children)):
    if type(model_children[i]) == nn.Conv2d:
        counter+=1
        model_weights.append(model_children[i].weight)
        conv_layers.append(model_children[i])
    elif type(model_children[i]) == nn.Sequential:
        for j in range(len(model_children[i])):
            for child in model_children[i][j].children():
                if type(child) == nn.Conv2d:
                    counter+=1
                    model_weights.append(child.weight)
                    conv_layers.append(child)
print(f"Total convolution layers: {counter}")
print(conv_layers)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [ ]:
image = transform(img)
image = image.unsqueeze(0)
image = image.to(device)

In [ ]:
outputs = []
names = []
for layer in conv_layers[0:]:
    image = layer(image)
    outputs.append(image)
    names.append(str(layer))
print(len(outputs))
#print feature_maps
for feature_map in outputs:
    print(feature_map.shape)

In [ ]:
processed = []
for feature_map in outputs:
    feature_map = feature_map.squeeze(0)
    gray_scale = torch.sum(feature_map,0)
    gray_scale = gray_scale / feature_map.shape[0]
    processed.append(gray_scale.data.cpu().numpy())
for fm in processed:
    print(fm.shape)

In [ ]:
outputs[0].shape

In [ ]:
fig = plt.figure(figsize=(30, 50))
for i in range(len(processed)):
    a = fig.add_subplot(10, 5, i+1)
    imgplot = plt.imshow(processed[i])
    a.axis("off")
    a.set_title(names[i].split('(')[0], fontsize=30)
plt.savefig(str('feature_maps.jpg'), bbox_inches='tight')

In [ ]:
processed[5].shape

In [ ]:
outputs[10].size()

In [ ]:
import torch.nn as nn

class SelfAttention(nn.Module):
    """ Self attention Layer"""
    def __init__(self,in_dim):
        super(SelfAttention,self).__init__()
        self.chanel_in = in_dim
        # self.activation = activation
        
        self.query_conv = nn.Conv2d(in_channels = in_dim , out_channels = in_dim//8 , kernel_size= 1)
        self.key_conv = nn.Conv2d(in_channels = in_dim , out_channels = in_dim//8 , kernel_size= 1)
        self.value_conv = nn.Conv2d(in_channels = in_dim , out_channels = in_dim , kernel_size= 1)
        self.gamma = nn.Parameter(torch.zeros(1))

        self.softmax  = nn.Softmax(dim=-1) #
    def forward(self,x):
        """
            inputs :
                x : input feature maps( B X C X W X H)
            returns :
                out : self attention value + input feature 
                attention: B X N X N (N is Width*Height)
        """
        m_batchsize,C,width ,height = x.size()
        proj_query  = self.query_conv(x).view(m_batchsize,-1,width*height).permute(0,2,1) # B X CX(N)
        proj_key =  self.key_conv(x).view(m_batchsize,-1,width*height) # B X C x (*W*H)
        energy =  torch.bmm(proj_query,proj_key) # transpose check
        attention = self.softmax(energy) # BX (N) X (N) 
        proj_value = self.value_conv(x).view(m_batchsize,-1,width*height) # B X C X N

        out = torch.bmm(proj_value,attention.permute(0,2,1) )
        out = out.view(m_batchsize,C,width,height)
        
        out = self.gamma*out + x
        return out,attention

In [ ]:
attn = SelfAttention(in_dim=512)

In [ ]:
outputs[15].size()

In [ ]:
out,attention = attn.forward(outputs[15].cpu())

In [ ]:
attention.squeeze().shape

In [ ]:
len(attention_maps)

In [ ]:
plt.imshow(attention.squeeze().detach().numpy())

In [ ]:
1, 2048, 14, 14

In [ ]:
import timm
from timm.models.layers.attention_pool2d import AttentionPool2d

In [ ]:
attn_timm = AttentionPool2d(in_features = feat_size=[2048, (14, 14)])

In [ ]:
import pandas as pd
import numpy as np
import random

In [ ]:
f1 = random.sample(range(0, 30), 5)
p = random.sample(range(0, 10), 5)
r = random.sample(range(0, 10), 5)


In [ ]:
print(f1, p, r)

In [ ]:
df = pd.DataFrame({'f1':f1,
                   'p':p,
                   'r':r})

In [ ]:
df

In [ ]:
dff = df.copy()

In [ ]:
dff = dff.append(r_di, ignore_index=True)

In [ ]:
dff

In [ ]:
r_di = {'f1':f1,
                   'p':p,
                   'r':r}

In [ ]:
r_di